# 1 — Load, làm sạch, và chia Train/Test

Notebook này:
1) Load `data/D11KS.csv`
2) Chuẩn hoá một số cột categorical và ép kiểu numeric
3) Làm sạch target `TenYearCHD`
4) Chia `train/test` theo **stratify (80/20)**

**Phụ thuộc:** đã chạy `app/00_setup.ipynb`.


In [ ]:
# 1) LOAD DATA (portable path)
from pathlib import Path

# Khuyến nghị: chạy Jupyter từ thư mục gốc của repo (nơi có README.md)
PROJECT_ROOT = Path.cwd()
DATA_PATH = PROJECT_ROOT / "data" / "D11KS.csv"

if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"Không tìm thấy '{DATA_PATH}'.\n"
        f"- Hãy tải dataset từ Kaggle và đặt vào: data/D11KS.csv\n"
        f"- Link: https://www.kaggle.com/datasets/phamphucai05/d11ks-csv"
    )

df = pd.read_csv(DATA_PATH)
display(df.head())
print("Shape:", df.shape)


## Kiểm tra nhanh cấu trúc dữ liệu

In [ ]:
# Kiểm tra nhanh cấu trúc dataset: danh sách cột, dtype từng cột, và cảnh báo nếu thiếu cột quan trọng

# In danh sách các cột hiện có
print("Columns:", list(df.columns))

# Hiển thị kiểu dữ liệu của từng cột (giúp phát hiện cột số bị đọc thành object/string)
print("\nDtypes:")
display(df.dtypes.to_frame("dtype"))

# Danh sách các cột kỳ vọng (schema chuẩn) cho bài toán dự đoán TenYearCHD
expected_cols = [
    "gender","age","education","currentSmoker","cigsPerDay","BPMeds","prevalentStroke",
    "prevalentHyp","diabetes","totChol","sysBP","diaBP","BMI","heartRate","glucose","TenYearCHD"
]

# Tìm các cột bị thiếu so với schema kỳ vọng
missing = [c for c in expected_cols if c not in df.columns]

# Nếu thiếu thì cảnh báo để bạn biết dataset không đúng cấu trúc mong đợi
if missing:
    print("WARNING: Missing expected columns:", missing)


## Chuẩn hoá categorical + ép kiểu numeric

In [ ]:
# Chuẩn hoá categorical (gender, yes/no) và ép kiểu numeric để dữ liệu sạch & đồng nhất trước modeling

def normalize_gender(x):
    # Chuẩn hoá cột gender về 2 nhãn: 'male' / 'female'; giá trị lạ -> NaN
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in {"m", "male", "nam", "1"}:
        return "male"
    if s in {"f", "female", "nu", "nữ", "0"}:
        return "female"
    return np.nan

def normalize_yesno(x):
    # Chuẩn hoá các cột dạng yes/no về 'Yes' / 'No'; giá trị lạ -> NaN
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in {"yes", "y", "true", "1"}:
        return "Yes"
    if s in {"no", "n", "false", "0"}:
        return "No"
    return np.nan

# Các cột text/categorical cần chuẩn hoá (nếu có trong df)
TEXT_COLS = ["gender", "currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes"]
for c in TEXT_COLS:
    # Bỏ qua nếu dataset không có cột đó
    if c not in df.columns:
        continue

    # gender dùng rule riêng; các cột còn lại dùng normalize_yesno
    if c == "gender":
        df[c] = df[c].apply(normalize_gender).astype("category")
    else:
        df[c] = df[c].apply(normalize_yesno).astype("category")

# Các cột numeric: ép về số, giá trị không parse được sẽ thành NaN
NUMERIC_COLS = ["age", "cigsPerDay", "totChol", "sysBP", "diaBP", "BMI", "heartRate", "glucose"]
for c in NUMERIC_COLS:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")


## Train/Test split (stratify) + làm sạch target

In [ ]:
# 9) Train/Test split with stratify (giữ tỷ lệ class 0/1) + làm sạch target để tránh lỗi NaN/inf
TARGET = "TenYearCHD"

# Làm sạch y: ép về numeric, chuyển inf/-inf -> NaN, rồi lấy index các dòng có target hợp lệ
y_raw = pd.to_numeric(df[TARGET], errors="coerce").replace([np.inf, -np.inf], np.nan)
valid_idx = y_raw.dropna().index

# Tạo X, y chỉ trên các dòng valid (tránh leak/lỗi do target NaN/inf)
X = df.loc[valid_idx].drop(columns=[TARGET])
y = y_raw.loc[valid_idx].astype(int)

# (Tuỳ chọn) Lọc y chỉ còn 0/1 nếu dữ liệu có giá trị khác
# mask = y.isin([0, 1])
# X, y = X.loc[mask], y.loc[mask]

# Chia train/test có stratify để giữ tỷ lệ dương/âm tương tự nhau ở cả 2 tập
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

# In kích thước và tỷ lệ dương tính (pos rate) để kiểm tra stratify hoạt động đúng
print("Train:", X_train.shape, "Test:", X_test.shape)
print("Train pos rate:", round(float(y_train.mean()), 4), "Test pos rate:", round(float(y_test.mean()), 4))
